In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
import sys
import time
import logging
import re
from commons import *
from gan_arch import *
from datetime import datetime

In [ ]:
IMG_W = 128
IMG_H = 128
IMG_C = 3


SHARDS = 1
BATCH_SIZE = 128
SUPER_BATCH = 8

latent_dim = 256
last_epoch = 0

SAMPLES_PATH = "samples_metu"
CHECKPOINT_PATH = "saved_model_metu"
LOG_DIR = "gan_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
GEN_CPT_NAME = f"{CHECKPOINT_PATH}/gencp-{{epoch:02d}}.ckpt"
DIS_CPT_NAME = f"{CHECKPOINT_PATH}/discp-{{epoch:02d}}.ckpt"

In [ ]:
METU_RAW_PATH = '/qarr/studia/magister/datasets/METU/930k_logo_v3/'
METU_DATASET_PATH = '/qarr/studia/magister/datasets/METU/resized_930k_logo'
METU_DATASET_PATH = '/home/zenfur/magister/resized_930k_logo'
EVAL_DATASET_PATH = "/home/zenfur/magister/eval_images"

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")

## Define METU dataset loading pipeline

In [ ]:
# Static load into RAM
# metuDataset = load_dataset(METU_DATASET_PATH, nexamples=1024*14)
# metuDataset = metuDataset.shuffle(len(metuDataset)).cache().batch(batch_size)

In [ ]:
imagesList = tf.io.matching_files(METU_DATASET_PATH + "/*.jpg")
print(f"Found {len(imagesList)} images in dataset")

In [ ]:
@tf.function
def tf_read_image(path):
    img = tf.io.read_file(path)
    return tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE')


@tf.function
def tf_convert_and_normalize_img(img):
    c = tf.constant(128.0, dtype=tf.dtypes.float32)
    img = tf.cast(img, tf.dtypes.float32)
    img = tf.math.subtract(img, c)
    return tf.math.divide(img, c)



In [ ]:
pathsDB = tf.data.Dataset.from_tensor_slices(imagesList)
lenMetu = len(pathsDB)
SHARD_IDX = 0
metuDB = (pathsDB.shard(SHARDS, SHARD_IDX)
                .shuffle(lenMetu//SHARDS + (lenMetu % SHARDS > SHARD_IDX))
                #.repeat()
                .map(tf_read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .batch(BATCH_SIZE*SUPER_BATCH)
                .map(tf_convert_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .prefetch(tf.data.experimental.AUTOTUNE)
                .unbatch()
                .batch(BATCH_SIZE)
         )

train_dataset = metuDB

In [ ]:
plt.imshow(tf_read_image(imagesList.numpy()[10]).numpy())

## Preparing the model

In [ ]:
# Attempting to read epoch number from last checkpoint
try:
    checkpoints = os.listdir(CHECKPOINT_PATH)
    last_epoch = np.max([int(re.search(r"-[0-9]+\.", i)[0][1:-1]) for i in checkpoints if
                             re.search(r"-[0-9]+\.", i)])
    print("Detected {} epoch as last checkpoint".format(last_epoch))
except (ValueError,  FileNotFoundError):
    last_epoch = 0
    print("Did not detect any checkpoints to continue from")

In [ ]:
# Initialize generator and disciminator models
weight_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
d_model = build_discriminator_ref(weight_init, IMG_H=IMG_H, IMG_W=IMG_W, IMG_C=IMG_C)
g_model = build_generator_ref(latent_dim, weight_init, IMG_H=IMG_H, IMG_W=IMG_W, IMG_C=IMG_C)

In [ ]:
if last_epoch:
    d_model.load_weights(DIS_CPT_NAME.format(epoch=last_epoch))
    g_model.load_weights(GEN_CPT_NAME.format(epoch=last_epoch))
    print(f"Loading models from epoch {last_epoch}")
else:
    print("No checkpoint to load from")

In [ ]:
gan = GAN_old(d_model, g_model, latent_dim)
bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
gan.compile(d_optimizer, g_optimizer, bce_loss_fn)

In [ ]:
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = LOG_DIR, histogram_freq=1)

In [ ]:
# Epochs to train
train_for = 2

In [ ]:
n_samples = 25
noise = np.random.normal(size=(n_samples, latent_dim))

logging.getLogger('tensorflow').setLevel(logging.ERROR)
try:
    for epoch in range(last_epoch, last_epoch+train_for):
        gan.fit(train_dataset, 
                initial_epoch=epoch, 
                epochs=epoch+1, 
                steps_per_epoch=lenMetu//SHARDS//BATCH_SIZE,
                callbacks=[tboard_callback]
               )
        g_model.save_weights(checkpoint_path_g.format(epoch=epoch+1))
        d_model.save_weights(checkpoint_path_d.format(epoch=epoch+1))

        n_samples = 25
        new_noise = np.random.normal(size=(n_samples, latent_dim))
        new_noise[0:5] = noise[0:5]
        examples = g_model.predict(new_noise)
        #save_plot(examples, epoch, int(np.sqrt(n_samples)))
        last_epoch += 1
except KeyboardInterrupt:
    print("Interrupted")

logging.getLogger('tensorflow').setLevel(logging.WARNING)

In [ ]:
tf.keras.backend.clear_session()

## Evaluating the trained model

Loading the evaluation dataset

In [ ]:
imagesList = tf.io.matching_files(EVAL_DATASET_PATH + "/*.jpg")

@tf.function
def tf_get_filename(path):
    return tf.strings.regex_replace(path, "[^/]*/", "")


#@tf.function
def tf_read_eval_image(path):
    # Retrieving the group number from file name
    img = tf.io.read_file(path)
    filename = tf_get_filename(path)
    group_number = tf.strings.to_number(
        tf.strings.regex_replace(filename, "-.*$", ""), 
        out_type=tf.dtypes.int32
    )
    return (tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE'), group_number)


#@tf.function
def tf_convert_and_normalize_img(img):
    c = tf.constant(128.0, dtype=tf.dtypes.float32)
    img = tf.cast(img, tf.dtypes.float32)
    img = tf.math.subtract(img, c)
    return tf.math.divide(img, c)


evalpathsDB = tf.data.Dataset.from_tensor_slices(imagesList)
DBlen = len(evalpathsDB)
convert_handler = lambda x,y:(tf_convert_and_normalize_img(x), y)
evalDB = (      evalpathsDB.map(tf_read_eval_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .batch(32)
                 .map(convert_handler, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .unbatch()
                 .cache()
)


In [ ]:
def generate_transformed_dataset(paths_database, mapping_function, batch=32):
    return (paths_database.map(tf_read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .batch(batch, drop_remainder=False)
                    .map(tf_convert_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .map(mapping_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    #.unbatch()
             )

In [ ]:
@tf.autograph.experimental.do_not_convert
def dummy_mapping_function(x):
    return tf.random.normal((tf.shape(x)[0], 128))

In [ ]:
def tf_db_to_array(tfdb, tfdb_size):
    shape = tf.shape(next(iter(tfdb.take(1))))
    batch_size = shape[0]
    my_array = np.empty(tuple([tfdb_size, *shape[1:]]))
    for i, el in enumerate(progressBar(iter(tfdb), iterable_size = len(tfdb))): 
        t = i*batch_size
        my_array[t:t+el.shape[0], :] = el
    return my_array


In [ ]:
evalArray = tf_db_to_array(generate_transformed_dataset(evalpathsDB, dummy_mapping_function), len(evalpathsDB))


In [ ]:
trainArray = tf_db_to_array(generate_transformed_dataset(pathsDB, dummy_mapping_function), len(pathsDB))

In [ ]:
def calculate_distances(origin, array, distance_norm):
    map_f = distance_norm(origin)
    return map_f(array)

In [ ]:
def euclidean_sq(origin):
    def euclidean_sq_map(point):
        return np.sum(np.power(point-origin,2), axis=-1)
    return euclidean_sq_map

In [ ]:
def cosine(origin):
    origin_norm = np.linalg.norm(origin)
    def cosine_map(point):
        return 1.0 - np.dot(point, origin) / np.linalg.norm(point, axis=-1) / origin_norm
    return cosine_map

In [ ]:
def arg_binsearch(array, x, cond):
    b = -1
    e = len(array)
    while e - b > 1:
        middle = (e+b)//2
        if cond(array[middle], x):
            b = middle
        else:
            e = middle
    return b

In [ ]:
class ProxyArray:
    def __init__(self, origin, proxy):
        self.origin = origin
        self.proxy = proxy
    
    def __getitem__(self, idx):
        return self.origin[self.proxy[idx]]
    
    def __len__(self):
        return len(self.origin)

In [ ]:
def get_ranks(trainArray, evalArray, x, distance_norm):
    '''
    Returns rankings table of evalDistX distances in trainDistX sorted order
    '''
    trainDistX = calculate_distances(x, trainArray, distance_norm)
    evalDistX = calculate_distances(x, evalArray, distance_norm)
    argSortTrain = np.argsort(trainDistX)
    proxy = ProxyArray(trainDistX, argSortTrain)
    
    return np.array(
        [(arg_binsearch(proxy, dist, np.less) + arg_binsearch(proxy, dist, np.less_equal))/2 for dist in evalDistX]
    )
    

In [ ]:
groupID = [j.numpy() for i,j in iter(evalDB)]

In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements 
    '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [ ]:
groups = {i:get_index_positions(groupID, i) for i in np.unique(groupID)}

In [ ]:
def calculate_rank(trainArray, evalArray, evalGroups, distance_norm):
    progbarsteps = (i for i in range(len(evalArray)))
    pbar = progressBar(progbarsteps, iterable_size = len(evalArray))
    results = {}
    for group, groupIdxs in evalGroups.items():
        for idx in groupIdxs:
            ranks = get_ranks(trainArray, evalArray, evalArray[idx], distance_norm)
            relevant_ranks = np.array([ranks[i] for i in groupIdxs if i != idx])
            results[idx] = relevant_ranks
            next(pbar)
    return results

In [ ]:
result = calculate_rank(trainArray, evalArray, groups, euclidean_sq)

In [ ]:
result = [result[i] for i in range(len(result))]

In [ ]:
result

In [ ]:
ranks = [np.mean(x) for x in result if x is not []]

In [ ]:
ranks_normalised = [(np.mean(x)-len(x)*(len(x)+1)/2)/len(trainArray) for x in result if x is not []]

In [ ]:
ranks

In [ ]:
ranks_normalised

In [ ]:
np.mean(ranks_normalised)

In [ ]:
2*np.mean(ranks)